### torchtext 라이브러리로 텍스트 분류 <hr>
- 1단계 - 데이터 전처리 : 숫자형식으로 변환하는 것 까지
- 2단계 - 모델 구현

[1-1] 데이터 준비 => 내장 데이터셋 활용
- AG_NEWS 데이터셋 반복자 : 라벨(label) + 문장의 튜플(tuple) 형태 

In [10]:
import torch
from torchtext.datasets import AG_NEWS

# DataPipe 타입 >>> iterator 타입 형변환
train_iter = iter(AG_NEWS(split='train'))

In [11]:
# !pip install torchdata

In [12]:
# 데이터 확인 => (label, text), label 1 ~ 4
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [13]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# 토커나이즈 생성
tokenizer = get_tokenizer('basic_english')

# 뉴스 학습 데이터 추출
train_iter = AG_NEWS(split='train')

In [14]:
## => 토큰 생성 제너레이터 함수 : 데이터 추출하여 토큰화
def yield_tokens(data_iter):
    # 라벨, 텍스트 -> 텍스트 토큰화( 문장마다 들어가는 단어를 쪼갠 것 )
    for _, txt in data_iter:
        yield tokenizer(txt)

In [15]:
## 단어사전 생성
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<UNK>"]) # min_freq = 기본값 : 1

## <UNK> 인덱스 0으로 설정
vocab.set_default_index(vocab["<UNK>"])

In [16]:
vocab(['<UNK>','here','is',' an','example'])
# 단어사전에다가 단어를 넣었을때 인덱스 반환

[0, 475, 21, 0, 5297]

In [17]:
# 텍스트 >> 정수 인코딩
text_pipeline = lambda x:vocab(tokenizer(x))

# 레이블 >> 정수 인코딩( 0 ~ 3 )
label_pipeline = lambda x: int(x) - 1

In [18]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 배치 크기만큼 데이터셋 반환 함수
def collate_batch(batch):
    '''
    batch : (label, text)
    '''
    # 배치 크기만큼의 라벨 텍스트 오프셋 값 저장 변수
    label_list, text_list, offsets = [], [], [0] # offset : 첫번째 데이터의 갯수
    
    # 뉴스기사 1개씩 라벨 추출해서 저장
    for (_label, _text) in batch:
        
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))

        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)

        # 텍스트 offset 즉, 텍스트 크기/길이 저장
        offsets.append(processed_text.size(0))

    # 텐서화 진행
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0) # 누적값 -> 전체 갯수
    text_list = torch.cat(text_list)
    
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [19]:
train_iter = AG_NEWS(split = 'train')
dataloader = DataLoader(train_iter,
                        batch_size = 8, # 기사를 8개 뽑아온다는 뜻 -> 각 기사마다 길이가 모두 다름
                        shuffle = False,
                        collate_fn = collate_batch)

In [20]:
# 분류 클래스 수와 단어사전 개수
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)

print(f'num_class : {num_class}   vocab_size : {vocab_size}')

num_class : 4   vocab_size : 95811


In [21]:
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR

In [31]:
# 모델 설계
# 입력층 : EmbeddingBag Layer - 레이어와 분류 목적을 위한 선형 레이어, 텍스트 길이는 오프셋
# 은닉층 : Linear - 4개 클래스 분류
class TextModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_class):
        super(TextModel, self).__init__()
        # 모델 구성 층 정의 - 패딩을 안했을때 EmbeddingBag 사용
        self.embedding = nn.EmbeddingBag(vocab_size, embedding_dim, sparse=False)
        self.fc = nn.Linear(embedding_dim, num_class)
        self.init_weights()

    # 가중치 초기화
    def init_weights(self):
        initrange = 0.5
        # uniform : 균등분포
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    # 순방향 학습 진행
    def forward(self,text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)        

In [32]:
HIDDEN_SIZE = 3
EMBEDDING_DIM = 64
VOCAB_SIZE = len(vocab)
NUM_CLASS = len(set([label for (label, text) in train_iter]))
EPOCHS = 10
LR = 5
BATCH_SIZE = 64

In [33]:
# 학습 관련 인스턴스
MODEL = TextModel(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_SIZE, NUM_CLASS).to(device)

CRITERION = nn.CrossEntropyLoss()
OPTIMIZER = torch.optim.SGD(MODEL.parameters(), lr=LR)
SCHEDULER = torch.optim.lr_scheduler.StepLR(OPTIMIZER, 1.0, gamma=0.1) # gamma : step마다 lr을 줄이는 간격

학습관련 함수들

In [34]:
# 학습 관련 함수 정의 


def train(model, dataloader, optimizer, criterion, epoch):
    
    model.train()
    
    # 학습 평가 관련 변수들
    total_acc, total_count = 0,0
    log_interval=300
    
    for idx, (label, text, offsets) in enumerate(dataloader):
        predicted_label = model(text, offsets)
        optimizer.zero_grad()
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        
        # 배치 학습 평가
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        
        if idx % log_interval == 0 and idx > 0:
            print(f"epoch : {epoch} batch : {idx} loss : {loss.item()}")
            print(f"Accuracy : {total_acc/total_count}")
            total_acc, total_count = 0,0
            
            break

In [35]:
def evaluate(model, dataloader, criterion):
    model.eval()
    
    total_acc, total_count = 0,0
    
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            
    return total_acc/total_count

In [36]:
def predict(model, text, text_pipeline):
    with torch.no_grad():
        # 토큰화 > 정수 변환 > 텐서
        text = torch.tensor(text_pipeline(text), dtype=torch.int64).to(device)
        text = text.unsqueeze(0)
        offsets = torch.tensor([0]).to(device)
        predicted_label = model(text, offsets)
        return predicted_label.argmax(1).item() + 1

- 학습 진행

In [37]:
# 학습 및 검증 진행
for epoch in range(1, EPOCHS+1):
    train(MODEL, dataloader, OPTIMIZER, CRITERION, epoch)
    accu_val = evaluate(MODEL, dataloader, CRITERION)
    print(f"epoch : {epoch} Accuracy : {accu_val}")
    SCHEDULER.step()

epoch : 1 batch : 300 loss : 1.052221655845642
Accuracy : 0.4676079734219269


c:\Users\mybae\anaconda3\envs\TORCH17_NLP38\lib\site-packages\torch\utils\data\datapipes\iter\combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


epoch : 1 Accuracy : 0.429925
epoch : 2 batch : 300 loss : 1.0132092237472534
Accuracy : 0.6000830564784053
epoch : 2 Accuracy : 0.5110666666666667
epoch : 3 batch : 300 loss : 0.9934564828872681
Accuracy : 0.6258305647840532
epoch : 3 Accuracy : 0.521375
epoch : 4 batch : 300 loss : 0.9926716685295105
Accuracy : 0.651578073089701
epoch : 4 Accuracy : 0.5203
epoch : 5 batch : 300 loss : 0.9926028251647949
Accuracy : 0.6503322259136213
epoch : 5 Accuracy : 0.5202583333333334
epoch : 6 batch : 300 loss : 0.9925960302352905
Accuracy : 0.6503322259136213
epoch : 6 Accuracy : 0.5202
epoch : 7 batch : 300 loss : 0.9925957322120667
Accuracy : 0.6503322259136213
epoch : 7 Accuracy : 0.5201916666666667
epoch : 8 batch : 300 loss : 0.9925956130027771
Accuracy : 0.6503322259136213
epoch : 8 Accuracy : 0.5201916666666667
epoch : 9 batch : 300 loss : 0.9925955533981323
Accuracy : 0.6503322259136213
epoch : 9 Accuracy : 0.5201916666666667
epoch : 10 batch : 300 loss : 0.9925955533981323
Accuracy : 0